<a href="https://colab.research.google.com/github/Abhishek-9938/Emotion-Recognition-and-Detection/blob/main/Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Helper Function

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-09-22 11:42:20--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-22 11:42:20 (113 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
from helper_functions import load_and_prep_image,create_tensorboard_callback,unzip_data
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential

# Unzipping the Data

In [ ]:
unzip_data("/content/drive/MyDrive/archive.zip")


#Preprocessing the train data

In [ ]:
train_data_gen=ImageDataGenerator(rescale=1/255.,
                                  rotation_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)
val_data_gen=ImageDataGenerator(rescale=1/255.)

In [ ]:
train_dir="/content/train"
test_dir="/content/test"

In [ ]:
train_data=train_data_gen.flow_from_directory(train_dir,
                                              target_size=(48,48),

                                              batch_size=32,
                                              seed=42)
val_data=val_data_gen.flow_from_directory(test_dir,
                                          target_size=(48,48),

                                          batch_size=32,
                                          shuffle=False)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
class_names=["Angry","Disquist","Fear","Happy","Neutral","Sad","Surprise"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Building our model

In [ ]:
model2=tf.keras.Sequential([
    layers.Conv2D(
            filters=64,
            kernel_size=(5,5),
            input_shape=(48,48,1),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_1'
    ),
    layers.BatchNormalization(name='batchnorm_1'),

    layers.Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'
    ),
    layers.BatchNormalization(name='batchnorm_2'),
    layers.MaxPool2D(pool_size=(2,2), name='maxpool2d_1'),
    layers.Dropout(0.4, name='dropout_1'),

    layers.Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_3'
    ),
    layers.BatchNormalization(name='batchnorm_3'),

    layers.Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_4'
    ),
    layers.BatchNormalization(name='batchnorm_4'),
    layers.MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'),
    layers.Dropout(0.4, name='dropout_2'),

    layers.Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_5'
        ),
    layers.BatchNormalization(name='batchnorm_5'),

    layers.Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_6'
        ),
    layers.BatchNormalization(name='batchnorm_6'),
    layers.MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'),
    layers.Dropout(0.5, name='dropout_3'),
    layers.Flatten(name='flatten'),

    Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal',
            name='dense_1'
         ),
    layers.BatchNormalization(name='batchnorm_7'),
    layers.Dropout(0.6, name='dropout_4'),
    Dense(
            len(class_names),
            activation='softmax',
            name='out_layer'
         )
])



In [ ]:

model.compile(loss="categorical_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [ ]:
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 48, 48, 3)]       0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 pooling_layer (GlobalAvera  (None, 512)               0         
 gePooling2D)                                                    
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
 softmax_float32 (Activatio  (None, 7)                 0         
 n)                                                              
                                                                 
Total params: 20027975 (76.40 MB)
Trainable params: 2002797

# Adding Model Checkpoint Callback and Early stopping callback

In [ ]:

## having early stopping and model check point

from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau

# early stopping
es = EarlyStopping(monitor='val_accuracy', min_delta= 0.00005 , patience= 11, verbose= 0, mode='auto')

# model check point
mc = ModelCheckpoint(filepath="best5_model.h5", monitor= 'val_accuracy', verbose= 0, save_best_only= True, mode = 'auto')

sch = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

# puting call back in a list
call_back = [es, mc]

In [ ]:
train_data.batch_size



32

In [ ]:

history=model.fit(train_data,
                   epochs=25,
                   steps_per_epoch=len(train_data),
                   validation_data=val_data,
                   callbacks=[es,mc,sch],
                   use_multiprocessing=True)

Epoch 1/25
898/898 [==============================] - 102s 104ms/step - loss: 1.8398 - accuracy: 0.2456 - val_loss: 1.8197 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 2/25
898/898 [==============================] - 92s 103ms/step - loss: 1.8150 - accuracy: 0.2511 - val_loss: 1.8205 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 3/25
898/898 [==============================] - 96s 106ms/step - loss: 1.8148 - accuracy: 0.2507 - val_loss: 1.8161 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 4/25
898/898 [==============================] - 96s 107ms/step - loss: 1.8131 - accuracy: 0.2513 - val_loss: 1.8149 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 5/25
898/898 [==============================] - 97s 108ms/step - loss: 1.8125 - accuracy: 0.2513 - val_loss: 1.8141 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 6/25
898/898 [==============================] - 98s 109ms/step - loss: 1.8124 - accuracy: 0.2513 - val_loss: 1.8136 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 7/25
898/898 [==============================]

In [ ]:
result=model.evaluate(val_data)

225/225 [==============================] - 4s 17ms/step - loss: 1.8135 - accuracy: 0.2471


# plotting the loss curve

In [ ]:
ck+ existing model
fer2013 vg19

In [ ]:
plt.subplot(1,2,1)
plt.title("accuracy vs val_accuracy")
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"],color="green")

plt.subplot(1,2,2)
plt.title("loss vs val_loss")
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"],color="red")

plt.show();